# Neural Network - Abalone Age by Classification with Bins

# Project 3 - Predicting the Age of Abalone

In [1]:
# Dependancies

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense

## Data Preprocessing

In [2]:
# Read in unscaled version of the dataset
unscaled_df = pd.read_csv('./Dataset/abalone_unscaled_data.csv')
unscaled_df.head()

,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings,Age
0,M,91,73,19,102.8,44.9,20.2,30.0,15,16.5
1,M,70,53,18,45.1,19.9,9.7,14.0,7,8.5
2,F,106,84,27,135.4,51.3,28.3,42.0,9,10.5
3,M,88,73,25,103.2,43.1,22.8,31.0,10,11.5
4,I,66,51,16,41.0,17.9,7.9,11.0,7,8.5


In [3]:
# Label Encode the categorical "Sex" column
# Note: Female => 0
#       Infant => 1
#       Male   => 2
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
unscaled_df['Sex_LabelEncoded'] = le.fit_transform(unscaled_df['Sex'])
unscaled_df.head()


,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings,Age,Sex_LabelEncoded
0,M,91,73,19,102.8,44.9,20.2,30.0,15,16.5,2
1,M,70,53,18,45.1,19.9,9.7,14.0,7,8.5,2
2,F,106,84,27,135.4,51.3,28.3,42.0,9,10.5,0
3,M,88,73,25,103.2,43.1,22.8,31.0,10,11.5,2
4,I,66,51,16,41.0,17.9,7.9,11.0,7,8.5,1


In [4]:
# Binary encode the categorical "Sex" column
unscaled_df = pd.get_dummies(unscaled_df, prefix=['Sex'], columns=['Sex'])
unscaled_df.head()

,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings,Age,Sex_LabelEncoded,Sex_F,Sex_I,Sex_M
0,91,73,19,102.8,44.9,20.2,30.0,15,16.5,2,0,0,1
1,70,53,18,45.1,19.9,9.7,14.0,7,8.5,2,0,0,1
2,106,84,27,135.4,51.3,28.3,42.0,9,10.5,0,1,0,0
3,88,73,25,103.2,43.1,22.8,31.0,10,11.5,2,0,0,1
4,66,51,16,41.0,17.9,7.9,11.0,7,8.5,1,0,1,0


In [5]:
# Reorganize columns
column_order = ["Sex_LabelEncoded", "Sex_M", "Sex_F", "Sex_I", "Length", 
                "Diameter", "Height", "Whole_weight", "Shucked_weight", 
                "Viscera_weight", "Shell_weight", "Rings", "Age"]
unscaled_df = unscaled_df.reindex(columns=column_order)
unscaled_df.head()

,Sex_LabelEncoded,Sex_M,Sex_F,Sex_I,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings,Age
0,2,1,0,0,91,73,19,102.8,44.9,20.2,30.0,15,16.5
1,2,1,0,0,70,53,18,45.1,19.9,9.7,14.0,7,8.5
2,0,0,1,0,106,84,27,135.4,51.3,28.3,42.0,9,10.5
3,2,1,0,0,88,73,25,103.2,43.1,22.8,31.0,10,11.5
4,1,0,0,1,66,51,16,41.0,17.9,7.9,11.0,7,8.5


## Determine X and y (Example using Sex_LabelEncoded and y=Rings)

In [6]:
#For the neural network classification, the categories must have one-hot encoding, not just label encoding
#That is why only the one-hot encoded columns for sex were included and not the other versions

# Determine X 
X = unscaled_df[['Sex_M', 'Sex_F', 'Sex_I', 'Length', 'Diameter', 'Height', 'Whole_weight', 
                 'Shucked_weight', 'Viscera_weight', 'Shell_weight']]
X

,Sex_M,Sex_F,Sex_I,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight
0,1,0,0,91,73,19,102.8,44.9,20.2,30.0
1,1,0,0,70,53,18,45.1,19.9,9.7,14.0
2,0,1,0,106,84,27,135.4,51.3,28.3,42.0
3,1,0,0,88,73,25,103.2,43.1,22.8,31.0
4,0,0,1,66,51,16,41.0,17.9,7.9,11.0
...,...,...,...,...,...,...,...,...,...,...
4172,0,1,0,113,90,33,177.4,74.0,47.8,49.8
4173,1,0,0,118,88,27,193.2,87.8,42.9,52.1
4174,1,0,0,120,95,41,235.2,105.1,57.5,61.6
4175,0,1,0,125,97,30,218.9,106.2,52.2,59.2


## Prepare Data for Stratification

In [7]:
#The one-hot encoding for the rings is done in later cells
# Here we are determining the classes with only one value and removing them so that stratification on the train-test split will work
# Stratification is necessary because measureing the training v testing requires the same number of classes

# Determine y
y = unscaled_df["Rings"]
y

0       15
1        7
2        9
3       10
4        7
        ..
4172    11
4173    10
4174     9
4175    10
4176    12
Name: Rings, Length: 4177, dtype: int64

In [8]:
y.max()

29

In [9]:
y.min()

1

In [10]:
pd.Series(y).unique().size

28

## Turn Number of Rings into Bins

In [11]:
unscaled_bins_df = unscaled_df.copy()

In [12]:
unscaled_bins_df['Rings'] = unscaled_bins_df['Rings'].map({1: "One to Five Rings", 2: "One to Five Rings", 3: "One to Five Rings", 4: "One to Five Rings", 5: "One to Five Rings", 6: "Six to Ten Rings", \
7: "Six to Ten Rings", 8:  "Six to Ten Rings", 9: "Six to Ten Rings", 10: "Six to Ten Rings", 11: "Eleven to Fifteen Rings", 12: "Eleven to Fifteen Rings", 13: "Eleven to Fifteen Rings", 14: "Eleven to Fifteen Rings", \
15: "Eleven to Fifteen Rings", 16: "Sixteen to Twenty Rings", 17: "Sixteen to Twenty Rings", 18: "Sixteen to Twenty Rings", 19: "Sixteen to Twenty Rings", 20: "Sixteen to Twenty Rings", 21: "Twenty-one to Twenty-Five Rings", \
22: "Twenty-one to Twenty-Five Rings", 23: "Twenty-one to Twenty-Five Rings", 24: "Twenty-one to Twenty-Five Rings", 25: "Twenty-one to Twenty-Five Rings", 26: "Twenty-six to Thirty Rings", 26: "Twenty-six to Thirty Rings", \
27: "Twenty-six to Thirty Rings", 28: "Twenty-six to Thirty Rings", 29: "Twenty-six to Thirty Rings", 30: "Twenty-six to Thirty Rings"})

In [13]:
unscaled_bins_df.head()

,Sex_LabelEncoded,Sex_M,Sex_F,Sex_I,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings,Age
0,2,1,0,0,91,73,19,102.8,44.9,20.2,30.0,Eleven to Fifteen Rings,16.5
1,2,1,0,0,70,53,18,45.1,19.9,9.7,14.0,Six to Ten Rings,8.5
2,0,0,1,0,106,84,27,135.4,51.3,28.3,42.0,Six to Ten Rings,10.5
3,2,1,0,0,88,73,25,103.2,43.1,22.8,31.0,Six to Ten Rings,11.5
4,1,0,0,1,66,51,16,41.0,17.9,7.9,11.0,Six to Ten Rings,8.5


In [14]:
unscaled_bins_df['Rings'].unique()

array(['Eleven to Fifteen Rings', 'Six to Ten Rings',
       'Sixteen to Twenty Rings', 'One to Five Rings',
       'Twenty-one to Twenty-Five Rings', 'Twenty-six to Thirty Rings'],
      dtype=object)

In [15]:
# Determine y WITH BINS
y = unscaled_bins_df["Rings"]
y

0       Eleven to Fifteen Rings
1              Six to Ten Rings
2              Six to Ten Rings
3              Six to Ten Rings
4              Six to Ten Rings
                 ...           
4172    Eleven to Fifteen Rings
4173           Six to Ten Rings
4174           Six to Ten Rings
4175           Six to Ten Rings
4176    Eleven to Fifteen Rings
Name: Rings, Length: 4177, dtype: object

In [16]:
a = np.array(y)
unique, counts = np.unique(a, return_counts=True)
tally = dict(zip(unique, counts))
tally

{'Eleven to Fifteen Rings': 1186,
 'One to Five Rings': 189,
 'Six to Ten Rings': 2541,
 'Sixteen to Twenty Rings': 225,
 'Twenty-one to Twenty-Five Rings': 32,
 'Twenty-six to Thirty Rings': 4}

### Handle Classes with Only a Single Value

In [17]:
# This is to record the classes that only have a single value
# Classes with single value have to be removed in order to do stratification

solo_classes=[];

for key in tally:
    if tally[key] == 1:
        solo_classes.append(key)
solo_classes

[]

In [18]:
# This removes the rows with the classes with the single values for both X and y
solo_indices=[];
for i in range(len(solo_classes)):
    print(i)
    j = pd.Series(y)[pd.Series(y) == solo_classes[i]].index
    print(j)
    solo_indices.append(j)

In [19]:
X.shape

(4177, 10)

In [20]:
for i in range(len(solo_indices)):
    print(i)
    X = X.drop(solo_indices[i])

In [21]:
X.shape

(4177, 10)

In [22]:
y.shape

(4177,)

In [23]:
for i in range(len(solo_indices)):
    print(i)
    y = y.drop(solo_indices[i])

In [24]:
y.shape

(4177,)

## Perform Train-Test-Split

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

#NOTE: -1 after y is needed to correct for zero-indexing (the zero column)
# You would never have a training value for the zero column and it would only add noise.

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)

# Step 2: Convert encoded labels to one-hot-encoding
y_categorical = to_categorical(encoded_y)

# Stratification is necessary to ensure that the test and training data are directly comparable
X_train, X_test, y_train, y_test = train_test_split(
    X, y_categorical, stratify=y, random_state=1)

In [26]:
y_train

array([[1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]], dtype=float32)

## Scale our data

In [27]:
# Using MinMaxScaler instead of StandardScaler because this is a classification problem with a limited range
# Sex columns are already one-hot encoded

X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Build a Simple Neural Network

In [28]:
# Create an empty sequential model
model = Sequential()

In [29]:
# The input layer has 10 columns
X_train_scaled.shape

(3132, 10)

In [30]:
# Add the first layer where the input dimensions are the 10 columns of the training data AND the first hidden layer are added in this line
model.add(Dense(128, activation='relu', input_dim=X_train.shape[1], kernel_regularizer=keras.regularizers.l1_l2(l1=0.01, l2=0.01)))

In [31]:
# The output layer has 22 columns that are one-hot encoded
y_train.shape

(3132, 6)

In [32]:
# Add output layer
model.add(Dense(y_train.shape[1], activation="softmax", kernel_regularizer=keras.regularizers.l1_l2(l1=0.01, l2=0.01)))

In [33]:
# Compile the model using categorical_crossentropy for the loss function, the adam optimizer,
# and add accuracy to the training metrics
model.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=['accuracy'])

In [34]:
# Use the training data to fit (train) the model
model.fit(
    X_train_scaled,
    y_train,
    epochs=1000,
    shuffle=True,
    verbose=2
)

Epoch 1/1000
98/98 - 0s - loss: 2.9405 - accuracy: 0.6070
Epoch 2/1000
98/98 - 0s - loss: 1.7299 - accuracy: 0.6082
Epoch 3/1000
98/98 - 0s - loss: 1.3528 - accuracy: 0.6082
Epoch 4/1000
98/98 - 0s - loss: 1.2678 - accuracy: 0.6082
Epoch 5/1000
98/98 - 0s - loss: 1.2259 - accuracy: 0.6082
Epoch 6/1000
98/98 - 0s - loss: 1.1974 - accuracy: 0.6082
Epoch 7/1000
98/98 - 0s - loss: 1.1741 - accuracy: 0.6082
Epoch 8/1000
98/98 - 0s - loss: 1.1562 - accuracy: 0.6082
Epoch 9/1000
98/98 - 0s - loss: 1.1421 - accuracy: 0.6082
Epoch 10/1000
98/98 - 0s - loss: 1.1314 - accuracy: 0.6082
Epoch 11/1000
98/98 - 0s - loss: 1.1218 - accuracy: 0.6082
Epoch 12/1000
98/98 - 0s - loss: 1.1130 - accuracy: 0.6082
Epoch 13/1000
98/98 - 0s - loss: 1.1063 - accuracy: 0.6082
Epoch 14/1000
98/98 - 0s - loss: 1.0990 - accuracy: 0.6082
Epoch 15/1000
98/98 - 0s - loss: 1.0930 - accuracy: 0.6082
Epoch 16/1000
98/98 - 0s - loss: 1.0867 - accuracy: 0.6082
Epoch 17/1000
98/98 - 0s - loss: 1.0825 - accuracy: 0.6082
Epoch 

# Save the Trained Model

In [35]:
# Save the model
model.save("Abalone_trained_NNClassBin_v1.h5")

# Evaluate the Model

In [36]:
# Load the model
from tensorflow.keras.models import load_model
model = load_model("Abalone_trained_NNClassBin_v1.h5")

In [37]:
y_test.shape

(1045, 6)

In [38]:
# Evaluate the model using the training data - Training CHECK
training_model_loss, training_model_accuracy = model.evaluate(X_train_scaled, y_train, verbose=2)
print(f"Training Loss: {training_model_loss}, Training Accuracy: {training_model_accuracy}")

98/98 - 0s - loss: 0.9767 - accuracy: 0.6421
Training Loss: 0.976697564125061, Training Accuracy: 0.6420817375183105


In [39]:
# Evaluate the model using the training data
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

33/33 - 0s - loss: 0.9923 - accuracy: 0.6421
Loss: 0.992281436920166, Accuracy: 0.6421052813529968
